In [1]:
import pandas as pd
import os
from datetime import timedelta, date, datetime
from sqlalchemy import create_engine
from sqlalchemy.types import NVARCHAR, DATE, FLOAT, VARCHAR, DATETIME

import config

data_dir = "data"

engine = create_engine(
    f"mysql+pymysql://{config.sql_credentials['user']}:{config.sql_credentials['password']}@{config.sql_credentials['host']}:{config.sql_credentials['port']}/{config.sql_credentials['db']}",
    pool_pre_ping=True)

conn = engine.connect()

In [2]:
training_query = '''
SELECT
    s.placement_ad_id,
    s.match_request_id,
    s.carer_id,
    sent_at,
    updated_at,
    sms_type,
    if(HOUR(TIMEDIFF(valid_from,sent_at))<8,1,0) as carer_applied_in_8hrs,
    DATEDIFF(start_date,sent_at) as lead_time,
    if(end_date IS NULL,1,0) as ongoing,
    min_provider_rate,
    one_off_payments_total,
    if(driving_requirements='LICENSE_NEEDED',1,0) as licence_needed,
    if(driving_requirements='CAR_NEEDED',1,0) as car_needed,
    if(placement_requirements regexp 'MOVING_HANDLING',1,0) as moving_handling,
    if(placement_requirements regexp 'DEMENTIA',1,0) as dementia,
    if(placement_requirements regexp 'MENTAL_HEALTH_ISSUES',1,0) as mental_health_issues,
    if(placement_requirements regexp 'HOIST',1,0) as hoist,
    if(placement_requirements regexp 'PARKINSONS',1,0) as parkinsons,
    if(placement_requirements regexp 'STROKE',1,0) as stroke,
    if(placement_requirements regexp 'ALZHEIMERS',1,0) as alzheimers,
    if(placement_requirements regexp 'STOMA',1,0) as stoma,
    if(placement_requirements regexp 'DIABETES',1,0) as diabetes,
    if(placement_requirements regexp 'PEG',1,0) as peg,
    if(has_wifi=1,1,0) as has_wifi,
    if(smoking=1,1,0) as smoking,
    if(has_pets='NO_PETS' OR has_pets='' OR has_pets IS NULL,0,1) as has_pets,
    if(h.service_type_id LIKE '%%2_RECIPIENTS%%',1,0) AS has_two_crs,
    latitude,
    longitude
FROM
     (
         SELECT *,SUBSTRING(SUBSTRING(placement_ad_id,6),1,CHAR_LENGTH(SUBSTRING(placement_ad_id,6))-5) as match_request_id
         FROM live_STATS_MATCHING_MATCH_REQUEST_CARER_SMS
    ) s
LEFT JOIN live_STATS_MATCHING_PLACEMENT_AD_APPLICATION a
    on (s.match_request_id = a.match_request_id AND s.carer_id = a.carer_id )
LEFT JOIN live_STATS_MATCHING_PLACEMENT_AD_HISTORY h
ON (s.placement_ad_id = h.placement_ad_id)
WHERE s.placement_ad_id IS NOT NULL AND
      s.placement_id IS NOT NULL AND
      sent_at<'2021-08-18' AND
      sent_at>'2019-10-01' AND
      sms_type in ('urgentMatchSent','slowMatchingSent','preferredPlacementSent','initialPromotionSent','returningCarerSent')
    AND (h.updated_at IS NULL
    OR h.updated_at = (
        SELECT MAX(h2.updated_at)
        FROM live_STATS_MATCHING_PLACEMENT_AD_HISTORY h2
        WHERE s.placement_ad_id = h2.placement_ad_id
#         AND sent_at>=h2.updated_at ??
    ))
'''

In [ ]:
training_data = pd.read_sql(training_query, conn)

In [ ]:
training_data.head()